## Format input dictionaries to Micheal

In [1]:
import pandas as pd
import numpy as np
import shapefile
import us

In [2]:
population_dir = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/population.csv'
count_pop_dir = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/county-pop.csv'
dist_matrix_dir = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/counties_distances.csv'
city_county_dir = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/selected_centers.csv'

In [3]:
pop_df = pd.read_csv(population_dir)
states = list(pop_df.state.unique())
states = [us.states.lookup(x).abbr for x in states]

In [4]:
county_pop_df = pd.read_csv(count_pop_dir)

county_pop_df.rename(columns={
    'STATE':'state_num', 
    'COUNTY': 'county_num',
    'CTYNAME':'county', 
    'STNAME':'state', 
}, inplace=True)

county_pop_df['county'] = [x.split(' ')[0].lower() for x in county_pop_df['county']]
county_pop_df['state'] = [us.states.lookup(x).abbr for x in county_pop_df['state']]

# Perform aggregation to match risk-class
county_pop_df['pop1049'] = county_pop_df.iloc[:, 3:7].sum(axis=1)
county_pop_df = county_pop_df.drop(columns=['pop19', 'pop29', 'pop39', 'pop49'])

cols_order = ['state_num', 'county_num', 'county', 'state', 'pop09', 'pop1049', 'pop59', 'pop69', 'pop79', 'pop89']
county_pop_df = county_pop_df[cols_order]

county_pop_df[cols_order]
county_pop_df['county_fips'] = 1000*county_pop_df['state_num'] + county_pop_df['county_num']
county_pop_df['county_state'] = county_pop_df['state'] + " " + county_pop_df['county_fips'].apply('{0:0>5}'.format)

In [5]:
county_pop_df.sort_values('county_state', inplace=True)
county_pop_df

,state_num,county_num,county,state,pop09,pop1049,pop59,pop69,pop79,pop89,county_fips,county_state
67,2,13,aleutians,AK,148,284,662,319,183,59,2013,AK 02013
68,2,16,aleutians,AK,370,732,1100,664,147,33,2016,AK 02016
69,2,20,anchorage,AK,39537,75780,35127,29417,14678,6245,2020,AK 02020
70,2,50,bethel,AK,3902,7113,1949,1478,631,185,2050,AK 02050
71,2,60,bristol,AK,104,168,136,121,51,30,2060,AK 02060
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,37,sweetwater,WY,5788,12049,4983,5026,2354,972,56037,WY 56037
3138,56,39,teton,WY,2307,4586,2895,2904,1688,697,56039,WY 56039
3139,56,41,uinta,WY,3012,6215,2315,2643,1297,527,56041,WY 56041
3140,56,43,washakie,WY,841,1960,1017,1120,749,453,56043,WY 56043


## Geolocation data

In [6]:
coords_dir = "/Users/alessandropreviero/Downloads/c_10nv20/c_10nv20.shp"
sf = shapefile.Reader(coords_dir)
num_counties = len(sf.records())
county_df_colnames = ['state', 'county','fips', 'fips_state',  'geolocation']
county_coords = []

for idx in range(num_counties):
    record = sf.record(idx)
    state_code = record[0]
    county_name = record[2].lower()
    county_fips = record[3]
    fips_state = record[0] + ' ' + county_fips
    lon = record[6]
    lat = record[7]
    county_coords.append([state_code, county_name, county_fips, fips_state, (lat, lon)])

counties_df = pd.DataFrame(county_coords, columns=county_df_colnames)
counties_df = counties_df[counties_df.state.isin(states)]

In [7]:
unique_pop = set(county_pop_df['county_state'].unique())
unique_counties = set(counties_df.fips_state.unique())

final_codes = list(set.intersection(unique_pop, unique_counties))

In [8]:
counties_df = counties_df[counties_df['fips_state'].isin(final_codes)]
counties_df.drop_duplicates('fips_state', inplace=True)

In [9]:
counties_df.sort_values('fips_state', inplace=True)
counties_df

,state,county,fips,fips_state,geolocation
3287,AK,aleutians east,02013,AK 02013,"(55.4039, -161.863)"
3288,AK,aleutians west,02016,AK 02016,"(52.8883, -111.1392)"
3298,AK,anchorage,02020,AK 02020,"(61.1501, -149.1067)"
3294,AK,bethel,02050,AK 02050,"(60.9126, -159.7207)"
3303,AK,bristol bay,02060,AK 02060,"(58.7423, -156.7013)"
...,...,...,...,...,...
2820,WY,sweetwater,56037,WY 56037,"(41.6595, -108.8794)"
2823,WY,teton,56039,WY 56039,"(43.9337, -110.5901)"
2721,WY,uinta,56041,WY 56041,"(41.2876, -110.5476)"
2827,WY,washakie,56043,WY 56043,"(43.905, -107.6829)"


In [10]:
counties_df.to_csv("counties_geo_clean.csv", index=False)

In [11]:
cols_order = ['state', 'county_state', 'pop09', 'pop1049', 'pop59', 'pop69', 'pop79', 'pop89']
county_pop_df = county_pop_df[cols_order]
county_pop_df = county_pop_df[county_pop_df.county_state.isin(final_codes)]
county_pop_df.sort_values('county_state', inplace=True)
county_pop_df.shape

(3141, 8)

In [13]:
county_pop_df.to_csv("county_pop_clean.csv", index=False)